# アニメデータの基礎分析

データ可視化の前に、以下のアニメデータの [**基礎分析**](basic-analysis) を行いましょう。

- `an_ae.csv`：アニメ各話についてまとめたファイル
- `an_ac_crt.csv`：アニメ作品と原作者の関係をまとめたファイル
- `an_ac_act.csv`：アニメ作品と声優の関係をまとめたファイル

なお、データの生成方法に関しては[アニメデータの前処理](../appendix/an_preprocess.ipynb)を参照ください。

## 初期設定

以降では、基礎分析に必要な初期設定を行います。
なお、紙幅の都合のため、可視化手法の説明に必須ではないソースコードは書籍版では割愛していますのでご注意ください。

### Import

必要なライブラリをImportします。

In [1]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [2]:
# pathlibモジュールのインポート
# ファイルシステムのパスを扱う
from pathlib import Path

# numpy：数値計算ライブラリのインポート
# npという名前で参照可能
import numpy as np

# pandas：データ解析ライブラリのインポート
# pdという名前で参照可能
import pandas as pd

# tqdm_notebookのインポート
# Jupyter Notebook環境でのプログレスバー表示に使用
# tqdmという名前で参照可能
from tqdm import tqdm_notebook as tqdm

### 定数

本Notebookで用いる定数を定義します。 なお、Pythonにおける定数の扱いについては、[こちら](python-const)を参照ください。

In [3]:
# 読み込み対象ディレクトリの定義

# アニメデータが保存されているディレクトリのパス
DIR_IN = Path("../../data/an/input")

In [4]:
# 読み込み対象ファイル名の定義

# アニメ各話に関するファイル
FN_AE = "an_ae.csv"

# アニメ作品と原作者の対応関係に関するファイル
FN_AC_CRT = "an_ac_crt.csv"

# アニメ作品と声優の対応関係に関するファイル
FN_AC_ACT = "an_ac_act.csv"

## `an_ae.csv`の基礎分析

`an_ae.csv`は、Anime Episodeに関する情報を一つに集約したファイルです。
次のような列を持ちます：

- `aeid`：Anime Episode ID。各話ID
- `aename`：Anime Episode NAME。各話名
- `date`：放送日
- `aeno`：各話数
- `acid`：Anime Collection ID。アニメ作品ID
- `acname`：Anime Collection NAME。アニメ作品名
- `asid`：Anime Series ID。アニメシリーズ名

In [5]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ae = pd.read_csv(DIR_IN / FN_AE)

### 全体像の把握

`df_ae`を概観してみましょう。

In [6]:
# df_aeデータフレームの先頭5行を転置して表示
df_ae.head().T

,0,1,2,3,4
aeid,M19760,M19761,M19762,M19763,M19764
aename,アトム誕生の巻＊,フランケンの巻＊,火星探険の巻＊,ゲルニカの巻＊,スフィンクスの巻＊
date,1963-01-01,1963-01-08,1963-01-15,1963-01-22,1963-01-29
aeno,第1話,第2話,第3話,第4話,第5話
acid,C7163,C7163,C7163,C7163,C7163
acname,鉄腕アトム,鉄腕アトム,鉄腕アトム,鉄腕アトム,鉄腕アトム
asid,C979,C979,C979,C979,C979


形状を確認してみましょう。

In [7]:
# df_aeデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ae.shape

(111041, 7)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [8]:
# df_aeデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算
df_ae.isna().agg([sum, "mean"])

,aeid,aename,date,aeno,acid,acname,asid
sum,0.0,2676.000000,0.0,606.000000,0.0,0.0,402.00000
mean,0.0,0.024099,0.0,0.005457,0.0,0.0,0.00362


`aename`に約2%程度、`aeno`に約0.5%程度、そして`asid`に約0.3%程度の欠損があることがわかります。

記述統計情報を確認してみましょう。

In [9]:
# df_aeデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の基本的な統計量を表示
df_ae.describe()

,aeid,aename,date,aeno,acid,acname,asid
count,111041,108365,111041,110435,111041,111041,110639
unique,111041,105955,10360,4844,3637,3631,2527
top,M19760,[総集編],2015-10-04,1,C8849,クレヨンしんちゃん,C1462
freq,1,68,66,3207,1926,1926,2084


ユニークな値の数を確認してみましょう。

In [10]:
# df_aeデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を`nunique`とする
df_ae.nunique().reset_index(name="nunique")

,index,nunique
0,aeid,111041
1,aename,105955
2,date,10360
3,aeno,4844
4,acid,3637
5,acname,3631
6,asid,2527


`aeid`、すなわちアニメ各話IDを主キーとするデータであることがわかります。
また`asid`は`acid`を束ねる上位概念であるため、ユニーク数が`acid`より少なくなっているのも想定通りです。

以降、少し踏み込んだ分析を行います。

### `date`列の深掘り

まず、`date`（放送日）について分析してみましょう。

In [11]:
# df_aeデータフレームの`date`列から最も古い日付と最も新しい日付を取得
# min()メソッドを使用して、最も古い日付を抽出
# max()メソッドを使用して、最も新しい日付を抽出
df_ae["date"].min(), df_ae["date"].max()

('1963-01-01', '2017-10-15')

このデータは1963年1月1日から2017年10月15日に放送されたアニメを対象としていることがわかります。

In [12]:
# `date`列を日付型に変換
# pd.to_datetime()メソッドを使用して、文字列型の日付を日付型(datetime)に変換
df_ae["date"] = pd.to_datetime(df_ae["date"])

# 年に関する情報を新しい`year`列として追加
# dt.yearを使用して、日付から年のみを取得
df_ae["year"] = df_ae["date"].dt.year

# 月に関する情報を新しい`month`列として追加
# dt.monthを使用して、日付から月のみを取得
df_ae["month"] = df_ae["date"].dt.month

# 曜日に関する情報を新しい`weekday`列として追加
# dt.weekdayを使用して、日付から曜日の情報を数値(0:月曜日, 6:日曜日)として取得
df_ae["weekday"] = df_ae["date"].dt.weekday

In [13]:
# `year`（年）ごとに各カラムのユニークな値の数を集計
# 具体的には、groupbyメソッドで`year`を基準にグループ化し、
# nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ae.groupby("year")[["month", "asid", "acid", "aeid"]].nunique().reset_index()

,year,month,asid,acid,aeid
0,1963,12,1,2,59
1,1964,12,1,2,99
2,1965,12,1,2,76
3,1966,12,1,1,45
4,1971,3,1,1,10
5,1972,3,1,1,13
6,1974,12,2,2,65
7,1975,3,1,1,13
8,1979,9,1,1,39
9,1980,1,1,1,4


`1967`年から`2017`年まで全ての月で継続的にデータが存在するわけではありません。
例えば、`1967`から`1970`年、`1973`年、`1976`から`1978`年、そして`1981`から`1989`年は一作もアニメデータが格納されていません。
また、それ以外の年も、12ヶ月全てのデータが存在するわけではありません。

この一因として、元データである[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)のアニメデータ[^json]において、放送日情報の欠損が多く見られることが挙げられます。
[折れ線グラフ](../08/line.ipynb)等で日付に関する情報が必須であることから、本書では[前処理](../appendix/an_preprocess.ipynb)として **日付情報のないアニメ作品・アニメ各話のデータを事前に除外** しています。これにより`an_ae.csv`から除外[^drop]されたレコードの情報は`../../data/an/interim/ae_dropped.csv`に一覧化してあります。

[^drop]: 厳密には、放送日とアニメ作品名を必須列として定め、そのいずれかが欠損している行を全て除外しました。詳細は[アニメデータの前処理](../appendix/an_preprocess.ipynb)を参照ください。

[^json]: 厳密には、[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)の`metadata_an-item_an201_json/*`において、`datePublished`が欠損しているレコードが多く見られます。

`ae_dropped.csv`には以下のような形式でアニメ各話が格納されています。

In [17]:
# headコマンドを用いて、ae_dropped.csvの先頭10行を確認
!head ../../data/an/interim/ae_dropped.csv

aeid,aename,date,aeno,acid,acname,asid
M817508,サラリーマン冒険者,2017-10-03,第1話,,,
M817557,美女と少年,2017-10-03,1,,,
M817593,第一話「さくやこのはな」,2017-10-05,1,,,
M817533,「猪も七代目には豚になる」,2017-10-06,1,,,
M817545,『人を殺すことができる国』,2017-10-06,1,,,
M817496,フォスフォフィライト,2017-10-07,1,,,
M817520,理由あって、アイドル！,2017-10-07,第1話,,,
M817581,ショコラウサギちゃんとみんなの夢,2017-10-07,1,,,
M817484,「私たちの大事な世界の全てだった」,2017-10-08,1,,,


`wc`コマンドを用いて行数（アニメ各話数）をカウントしてみましょう。

In [18]:
# wcコマンドを用いて、ae_dropped.csvの行数をカウント
!wc -l ../../data/an/interim/ae_dropped.csv

2155 ../../data/an/interim/ae_dropped.csv


`an_ae.csv`から除外されたアニメ作品として、例えば`シティーハンター`シリーズの一部があります。

In [15]:
# grepコマンドで、シティーハンターを含む文字列をae_dropped.csvから抽出
!grep シティーハンター ../../data/an/interim/ae_dropped.csv

M22564,,,,C8031,シティーハンター,C6942
M22642,,,,C8317,シティーハンター2,C6942
M38486,　　　　,,,C9275,シティーハンタースペシャル'96 ザ・シークレット・サービス,C6942


`an_ae.csv`に **全てのアニメ作品の情報が含まれているわけではない** ことは常に意識しておきましょう。

In [16]:
# `weekday`（曜日）ごとに各カラムのユニークな値の数を集計
# 具体的には、groupbyメソッドで`weekday`を基準にグループ化し、
# nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ae.groupby("weekday")[["asid", "acid", "aeid"]].nunique().reset_index()

,weekday,asid,acid,aeid
0,0,575,734,14554
1,1,583,773,13572
2,2,578,731,13181
3,3,572,738,13060
4,4,723,903,17066
5,5,656,834,19837
6,6,611,777,19771


Pandasにおける`weekday`は`0`が月曜を表し、`6`が日曜を表します。 この集計により、（少なくともこのデータ上では）全ての曜日でアニメの放送実績があることがわかります。 
また、アニメ作品（`acid`）数が最も多いのは金曜日（`4`）ですが、アニメ各話数が多いのは土曜（`5`）や日曜（`6`）であることがわかります。

### `acid`、`acname`列の深掘り

`acid`（アニメ作品ID）、そして`acname`（アニメ作品名）について分析してみましょう。

まず、`acid`と`acname`の対応関係を確認します。

In [17]:
# `acid`ごとにユニークな`acname`の数を集計し、その統計情報を取得
df_ae.groupby("acid")["acname"].nunique().describe().reset_index()

,index,acname
0,count,3637.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`acid`に対して、必ず一つの`acname`が対応していることがわかります。

In [18]:
# `acname`ごとにユニークな`acid`の数を集計し、その統計情報を取得
df_ae.groupby("acname")["acid"].nunique().describe().reset_index()

,index,acid
0,count,3631.000000
1,mean,1.001652
2,std,0.040622
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,2.000000


一方で、一つの`acname`に対して、最大で二つの`acid`が対応することがあるようです。
では、具体的にはどのようなアニメ作品が、複数の`acid`と紐付いているのでしょうか？

In [19]:
# `acname`ごとにユニークな`acid`の数を集計
df_tmp = df_ae.groupby("acname")["acid"].nunique().reset_index()

# `acid`が複数ある`acname`を抽出
df_tmp[df_tmp["acid"] > 1]

,acname,acid
9,100％ パスカル先生,2
82,BUZZER BEATER,2
202,Fate/Zero,2
739,うっかりペネロペ,2
975,じゃがいぬくん,2
3588,魔法陣グルグル,2


例えば、`100％ パスカル先生`について詳しく調べてみましょう。

In [20]:
# `df_ae`から`acname`が`100％ パスカル先生`である行を抽出
# 見やすいように特定の列を抽出
df_ae[df_ae["acname"] == "100％ パスカル先生"][["aename", "acid", "acname", "asid"]]

,aename,acid,acname,asid
107203,[第1話],C16019,100％ パスカル先生,C6536
107926,[第2話],C16019,100％ パスカル先生,C6536
109088,1時間目 その名もパスカル先生,C16134,100％ パスカル先生,C6536
109089,2時間目 完璧[パーフェクト]プレート,C16134,100％ パスカル先生,C6536
109184,1時間目 恐怖!100!身体検査,C16134,100％ パスカル先生,C6536
...,...,...,...,...
110958,2時間目 完璧[パーフェクト]プレート,C16134,100％ パスカル先生,C6536
110959,3時間目 明日[あした]使えるヒエログリフ講座,C16134,100％ パスカル先生,C6536
111014,1時間目 体育大魔王 襲来!!,C16134,100％ パスカル先生,C6536
111015,2時間目 完璧[パーフェクト]プレート,C16134,100％ パスカル先生,C6536


[Wikipedia](https://ja.wikipedia.org/wiki/100%25%E3%83%91%E3%82%B9%E3%82%AB%E3%83%AB%E5%85%88%E7%94%9F#%E3%83%86%E3%83%AC%E3%83%93%E3%82%A2%E3%83%8B%E3%83%A1)によると、アニメ`100％ パスカル先生`は二種類存在するようです。
一つはコロコロチャンネルで公開されていたWebアニメを「おはスタ」で放送したもの、もう一つはTBS系列全28局ネット「アニメサタデー630」第1部の前半（Aパート）にて放送されたものです。
前者が`C16019`、後者が`C16134`に該当することがわかりました。
ただし、`asid`としては両者とも`C6536`として管理されています。

次に、`acname`ごとの`aeid`数を集計してみます。

In [21]:
# `df_ae`で各アニメ作品(`acname`)ごとにユニークなアニメ各話(`aeid`)の数を集計
# その後、降順にソートして上位10件を表示
df_ae.groupby("acname")["aeid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,acname,aeid
0,クレヨンしんちゃん,1926
1,親子クラブ,1363
2,サザエさん,1175
3,ちびまる子ちゃん［新］,994
4,それいけ！アンパンマン,958
5,ONE PIECE,783
6,しましまとらの しまじろう,751
7,名探偵コナン,729
8,あたしンち,668
9,ドラえもん［新・第2期］,608


アニメ作品のうち、このデータに含まれる各話数が最も多いのは`クレヨンしんちゃん`であることがわかります。

次に、アニメ作品名の長さについて分析してみます。

In [22]:
# `df_ae`から、各アニメ作品ID(`acid`)ごとに
# 最初のアニメ作品名(`acname`)と年(`year`)を取得
df_tmp = df_ae.groupby("acid")[["acname", "year"]].first().reset_index()
# アニメ作品名(`acname`)の文字数を新しいカラム`l_acname`として追加
df_tmp["l_acname"] = df_tmp["acname"].str.len()

In [23]:
# アニメ作品名(`acname`)の文字数(`l_acname`)に関する統計量を取得
df_tmp["l_acname"].describe().reset_index()

,index,l_acname
0,count,3637.000000
1,mean,14.134177
2,std,9.486087
3,min,1.000000
4,25%,8.000000
5,50%,12.000000
6,75%,18.000000
7,max,167.000000


アニメ作品名は平均14文字、最大167文字であることがわかります。特にアニメ作品名が長いものを見てみます。

In [24]:
# アニメ作品名(`acname`)の文字数(`l_acname`)が長い上位10作品を取得
df_tmp.sort_values("l_acname", ascending=False).head(10)

,acid,acname,year,l_acname
2908,C16008,Occultic;Nine THERE ARE NO SUCH THING AS “OCCU...,2016,167
2218,C15007,"キャプテン･アース WHEN I OPENED THE DOOR CALLED TRUTH,...",2014,104
2394,C15395,四月は君の嘘 I met the girl under full-bloomed cher...,2014,93
2153,C14933,LOVELY❤ムービー いとしのムーコ Lovely Muuuuuuuco! The hap...,2013,88
2205,C14994,アオハライド THE SCENT OF AIR AFTER THE RAIN… I HEAR...,2014,76
2331,C15278,"ローリング☆ガールズ Rolling, Falling, Scrambling. For o...",2015,67
2843,C15917,アンジュ・ヴィエルジュ “Progress“:Girls facing destiny ag...,2016,66
2089,C14862,"戦姫絶唱 シンフォギアG In the distance,that day,when the...",2013,65
2449,C15451,艦隊これくしょん 艦 これ Fleet Girls Collection KanColle ...,2015,64
2629,C15667,コメットルシファーNOW ADVENTURE BEGINS-WITH YOU. FRIEND...,2015,63


`Occultic;Nine THERE ARE NO SUCH THING AS “OCCULT”. IT CAN BE DISPRIVED ALL BY SCIENCE. ONLY THE ONES WHO HAVE ACCEPTED EVERYTHING. CAN GET THE RIGHT TO KNOW THE TRUTH.`を始め、非常に長い英語の副題がついているアニメ作品が一定数存在することがわかります。

では、アニメ作品名の長さと初回放送年の関係はどのようになっているのでしょうか？

In [25]:
# 各初回放送年(`year`)ごとのアニメ作品名の文字数(`l_acname`)に関する統計量を集計
df_tmp.groupby("year")["l_acname"].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
1963,2.0,5.000000,0.000000,5.0,5.00,5.0,5.00,5.0
1971,1.0,5.000000,NaN,5.0,5.00,5.0,5.00,5.0
1974,2.0,8.500000,2.121320,7.0,7.75,8.5,9.25,10.0
1979,1.0,8.000000,NaN,8.0,8.00,8.0,8.00,8.0
1990,17.0,10.705882,3.869184,5.0,8.00,10.0,14.00,20.0
1991,37.0,11.891892,4.903422,5.0,8.00,11.0,16.00,23.0
1992,37.0,11.162162,4.645867,6.0,8.00,10.0,13.00,24.0
1993,24.0,9.625000,3.449165,4.0,8.50,9.0,11.25,19.0
1994,33.0,10.515152,5.166728,5.0,7.00,9.0,11.00,26.0


徐々にではありますが、放送開始年が新しくなるほど、アニメ作品名は長くなる傾向があるようです。
この理由を考察するのも面白いかもしれません。

### `asid`列の深堀り

`asid`（アニメシリーズID）について分析してみましょう。
まずは、`asid`と`acid`（アニメ作品ID）の包含関係を確認します。

In [26]:
# `asid`ごとにユニークな`acid`の数を集計し、その統計情報を取得
df_ae.groupby("asid")["acid"].nunique().describe().reset_index()

,index,acid
0,count,2527.000000
1,mean,1.436486
2,std,1.259258
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,26.000000


上記は、`asid`ごとのユニークな`acid`数に関する要約統計量です。
一つの`asid`に対して平均1.44、最大26の`acid`が紐づいています。

In [27]:
# `acid`ごとにユニークな`asid`の数を集計し、その統計情報を取得
# `asid`には欠損があるため事前に除外して集計
df_ae[~df_ae["asid"].isna()].groupby("acid")["asid"].nunique().describe().reset_index()

,index,asid
0,count,3630.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一方で上記は、`acid`ごとのユニークな`asid`数に関する要約統計量です。
一つの`acid`には、必ず一つの`asid`が紐づいています。
以上から、`asid`は`acid`を含む概念であることがわかりました。

少し気になったので、多くの`acid`と紐づいていてる`asid`を抽出してみましょう。

In [28]:
# df_aeデータフレームを'date'列で昇順に並び替えてから、'asid'でグループ化
# その後、各グループに対して'acid'と'aeid'のユニークな値の数を計算し、'acname'の最初の値を取得
# reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームdf_tmpとして取得
df_tmp = (
    df_ae.sort_values("date")
    .groupby("asid")
    .agg({"acid": "nunique", "aeid": "nunique", "acname": "first"})
    .reset_index()
)

# df_tmpデータフレームを'acid'列の値で降順に並び替えて、上位10行を表示
df_tmp.sort_values("acid", ascending=False).head(10)

,asid,acid,aeid,acname
98,C1462,26,2084,忍たま 乱太郎［第1期］
310,C2158,20,1653,おじゃる丸
2502,C6820,15,480,ビーストウォーズⅡ
1354,C4102,14,1245,いない いない ばあっ![第4期]
696,C2833,14,659,ふたりはプリキュア
597,C2650,13,551,デュエル・マスターズ
237,C1985,13,862,ポケットモンスター
1669,C4727,10,138,てーきゅう
1550,C4517,10,370,CARDFIGHT!! ヴァンガード
458,C2454,9,410,爆転シュート ベイブレード


本データセット中で最も多くのアニメ作品と紐づいているのは`asid`が`C1462`の`忍たま 乱太郎`シリーズであることがわかりました。
このシリーズに含まれるアニメ作品名を集計してみましょう。

In [29]:
# 'asid'が"C1462"の行だけを選択して、'acname'でグループ化した後、'aeid'のユニークな値の数と'date'の最小値を集計
# reset_index()で、結果のインデックスをリセットして、それを新しいデータフレームdf_nintamaとして取得
df_nintama = (
    df_ae[df_ae["asid"] == "C1462"]
    .groupby("acname")
    .agg({"aeid": "nunique", "date": "min"})
    .reset_index()
)

# df_nintamaデータフレームを'date'列で昇順に並び替え
# sort_values("date")で、'date'列を基準に昇順でデータを並び替え
df_nintama.sort_values("date")

,acname,aeid,date
10,忍たま 乱太郎［第1期］,95,1993-04-10
12,忍たま乱太郎 第2期,120,1994-10-03
17,忍たま乱太郎[第3期],120,1995-10-02
18,忍たま乱太郎[第4期],120,1996-04-01
19,忍たま乱太郎[第5期],100,1997-10-06
20,忍たま乱太郎[第6期],60,1998-04-06
21,忍たま乱太郎[第7期],80,1999-04-05
2,忍たま 乱太郎 [第8期],83,1999-12-31
25,忍たま乱太郎［第9期］,85,2000-12-23
22,忍たま乱太郎［第10期］,82,2002-03-21


`忍たま 乱太郎［第1期］`から`忍たま乱太郎[第25期]`までと、`忍たま 乱太郎 の 宇宙 大冒険 with コズミック フロント☆ N E X T`が`C1462`に含まれていました。

では、多くの`aeid`と紐づいている`asid`はどのようなものでしょうか。

In [30]:
# ユニークなaeid数に対して降順にソートし、上位10件を表示
df_tmp.sort_values("aeid", ascending=False).head(10)

,asid,acid,aeid,acname
98,C1462,26,2084,忍たま 乱太郎［第1期］
70,C1327,1,1926,クレヨンしんちゃん
310,C2158,20,1653,おじゃる丸
146,C1640,1,1363,親子クラブ
1354,C4102,14,1245,いない いない ばあっ![第4期]
112,C1497,4,1207,しましまとらの しまじろう
226,C1945,1,1175,サザエさん
0,C1022,1,994,ちびまる子ちゃん［新］
2515,C7048,1,958,それいけ！アンパンマン
283,C2116,8,920,遊☆戯☆王


最も多くのアニメ各話（`aeid`）と紐づいているのも`忍たま乱太郎`シリーズでした。
次に多かったのは`クレヨンしんちゃん`シリーズですが、こちらはアニメ作品数（`acid`）が1です。
シリーズによって、期やクールごとに別の`acid`を与えるかどうか異なることがわかります。

### `aeid`、`aename`、`aeno`の深堀り

`aeid`（アニメ各話ID）、`aename`（アニメ各話名）、`aeno`（アニメ各話数）について分析します。

まず、`aeid`と`aename`の対応関係を見てみましょう。

In [31]:
# `aeid`ごとにユニークな`aename`の数を集計し、その統計情報を取得
df_ae.groupby("aeid")["aename"].nunique().describe().reset_index()

,index,aename
0,count,111041.000000
1,mean,0.975901
2,std,0.153358
3,min,0.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,1.000000


一つの`aeid`に対して、平均約0.98の`aename`が紐づいていることがわかります。
1と一致しないのは、一部の行で`aename`が欠損していることが原因と考えられます。

In [32]:
# df_aeから`aename`が存在する列を抽出したうえで
# `aeid`ごとにユニークな`aename`の数を集計し、その統計情報を取得
df_ae[~df_ae["aename"].isna()].groupby("aeid")[
    "aename"
].nunique().describe().reset_index()

,index,aename
0,count,108365.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


`aename`が欠損している行を除外したうえで同様の集計を行ったところ、一つの`aeid`に対して必ず一つの`aename`が紐づいていることがわかりました。
では、逆はどうでしょうか？

In [33]:
# `aename`ごとにユニークな`aeid`の数を集計し、その統計情報を取得
df_ae.groupby("aename")["aeid"].nunique().describe().reset_index()

,index,aeid
0,count,105955.000000
1,mean,1.022746
2,std,0.332036
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,68.000000


一つの`aename`に対して、平均で約1、最大で68の`aeid`が紐づいていることがわかります。つまり、`aename`に関しては重複があるということです。
では具体的には、どのようなアニメ各話名が重複しているのでしょうか？

In [34]:
# `aename`ごとにユニークな`aeid`の数を集計
df_tmp = df_ae.groupby("aename")["aeid"].nunique().reset_index(name="n_ae")

# `n_ae`に関して降順ソートし、上位五つを表示
df_tmp.sort_values("n_ae", ascending=False).head()

,aename,n_ae
14994,[総集編],68
4167,2時間目 完璧[パーフェクト]プレート,25
29650,てんすけの ふるさとめぐり,25
13692,[サブタイトル表示なし],20
14713,[第1話],17


`[総集編]`、`[サブタイトル表示なし]`、そして`[第1話]`が重複するのは理解できます。
しかし、`2時間目 完璧[パーフェクト]プレート`（合計25話）や`てんすけの ふるさとめぐり`（合計25話）が重複することには違和感を感じます。
試しに、`aename`が`2時間目 完璧[パーフェクト]プレート`と一致するデータを抽出してみましょう。

In [35]:
# aenameが2時間目 完璧[パーフェクト]プレートと一致する行を抽出
# 見やすさのため、特定の列のみ表示
df_ae[df_ae["aename"] == "2時間目 完璧[パーフェクト]プレート"][
    ["date", "acname", "aeid", "aename"]
]

,date,acname,aeid,aename
109089,2017-04-15,100％ パスカル先生,M135357,2時間目 完璧[パーフェクト]プレート
109185,2017-04-22,100％ パスカル先生,M135359,2時間目 完璧[パーフェクト]プレート
109276,2017-04-29,100％ パスカル先生,M135362,2時間目 完璧[パーフェクト]プレート
109362,2017-05-06,100％ パスカル先生,M135364,2時間目 完璧[パーフェクト]プレート
109455,2017-05-13,100％ パスカル先生,M135367,2時間目 完璧[パーフェクト]プレート
109546,2017-05-20,100％ パスカル先生,M135369,2時間目 完璧[パーフェクト]プレート
109632,2017-05-27,100％ パスカル先生,M135372,2時間目 完璧[パーフェクト]プレート
109717,2017-06-03,100％ パスカル先生,M135374,2時間目 完璧[パーフェクト]プレート
109809,2017-06-10,100％ パスカル先生,M135377,2時間目 完璧[パーフェクト]プレート
109895,2017-06-17,100％ パスカル先生,M135380,2時間目 完璧[パーフェクト]プレート


アニメ作品`100％ パスカル先生`にて、2017年4月15日から2017年9月30日まで合計25回`2時間目 完璧[パーフェクト]プレート`が放送された実績があることがわかりました。
このアニメ作品では、各放送枠が複数パートから構成されており、毎回`2時間目 完璧[パーフェクト]プレート`というパートが存在した、と考えるのが自然でしょう。

## `an_ac_crt.csv`の基礎分析

`an_ac_crt.csv`は、アニメ作品と原作者に関する情報を一つに集約したファイルです。次のような列を持ちます：

- `acid`：Anime Collection ID。アニメ作品ID
- `acname`：Anime Collection NAME。アニメ作品名
- `asid`：Anime Series ID。アニメシリーズID
- `n_ae`：アニメ各話の合計数
- `first_date`：最初の放送日
- `last_date`：最後の放送日
- `crtid`：CReaTor ID。原作者ID
- `crtname`：CReaTor NAME。原作者名

In [36]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ac_crt = pd.read_csv(DIR_IN / FN_AC_CRT)

### 全体像の把握

まず、`df_ac_crt`を概観してみましょう。

In [37]:
# df_ac_crtデータフレームの先頭5行を転置して表示
df_ac_crt.head().T

,0,1,2,3,4
acid,C10010,C12657,C12663,C12681,C13191
acname,グラビテーション,ヒピラくん 原作/大友克洋,カウボーイ ビバップ[WOWOW放送版],ドラえもん［新］,HUNTER × HUNTER[新]
asid,C2336,C3943,C2111,NaN,C2136
n_ae,13,10,26,224,149
first_date,2000-10-04,2009-12-21,1998-10-24,1999-12-03,2011-10-02
last_date,2001-01-10,2009-12-24,1999-04-24,2005-03-18,2014-09-24
crtid,ACRT00944,ACRT00733,ACRT01173,ACRT01283,ACRT00647
crtname,村上真紀,大友克洋,矢立肇,藤子・F・不二雄,冨樫義博


形状を確認してみましょう。

In [38]:
# df_ac_crtデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ac_crt.shape

(1494, 8)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [39]:
# df_ac_crtデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算
df_ac_crt.isna().agg([sum, "mean"])

,acid,acname,asid,n_ae,first_date,last_date,crtid,crtname
sum,0.0,0.0,2.000000,0.0,0.0,0.0,0.0,0.0
mean,0.0,0.0,0.001339,0.0,0.0,0.0,0.0,0.0


`asid`に僅かな欠損があります。

記述統計情報を確認してみましょう。

In [40]:
# df_ac_crtデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の中央値、平均、標準偏差などの基本的な統計量を表示
df_ac_crt.describe()

,n_ae
count,1494.000000
mean,27.911647
std,47.584073
min,1.000000
25%,12.000000
50%,13.000000
75%,27.000000
max,1175.000000


ユニークな値の数を確認してみましょう。

In [41]:
# df_ac_crtデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を"nunique"とする
df_ac_crt.nunique().reset_index(name="nunique")

,index,nunique
0,acid,1107
1,acname,1106
2,asid,896
3,n_ae,111
4,first_date,545
5,last_date,580
6,crtid,1056
7,crtname,1056


行数より`acid`や`crtid`のユニーク数が少ないので、それぞれ単独では重複が存在することがわかります。

In [42]:
# `acid`と`crtid`の組み合わせで重複する行の数をカウント
df_ac_crt.duplicated(subset=["acid", "crtid"]).sum()

0

`acid`と`crtid`の組合せに関して重複がないことがわかります。

`acid`のカバー率（`an_ae.csv`の`acid`のうち、`an_crt_ac.csv`に存在するものの割合）を確認します。

In [43]:
# `df_ac_crt`と`df_ae`で共通の`ccid`の数を取得
n_acid_w_crt = len(set(df_ac_crt["acid"].unique()) & set(df_ae["acid"].unique()))

# `df_ae`内のユニークな`acid`の数を取得
n_acid = df_ae["acid"].nunique()

# カバー率を計算して表示
print(f"{n_acid_w_crt} / {n_acid} = {n_acid_w_crt / n_acid}")

1107 / 3637 = 0.30437173494638436


全体の約30%、1107のアニメ作品にのみ原作者が対応付けられていることがわかりました。

以降、カラムごとに少し踏み込んだ分析を行います。

### `acid`、`acname`列の深掘り

まず、`acid`（アニメ作品ID）と`acname`（アニメ作品名）について分析してみましょう。

In [44]:
# `acid`と`acname`ごとにユニークな`crtid`の数を集計し、結果の統計情報を取得
df_ac_crt.groupby(["acid", "acname"])["crtid"].nunique().describe().reset_index()

,index,crtid
0,count,1107.000000
1,mean,1.349593
2,std,0.573454
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,2.000000
7,max,4.000000


一つのアニメ作品に対して、平均で1.3人、最大で4人の原作者が紐付けられていることがわかります。

In [45]:
# `acid`と`acname`ごとにユニークな`crtid`の数を集計
# その後、降順にソートして、紐づいている原作者が多いアニメ作品を上位に表示
df_ac_crt.groupby(["acid", "acname"])["crtid"].nunique().sort_values(
    ascending=False
).reset_index().head()

,acid,acname,crtid
0,C16041,ぼのぼの[新],4
1,C15430,てさぐれ! 部活もの すぴんおふ プルプルんシャルム と遊ぼう,4
2,C16202,トミカハイパーレスキュー ドライブヘッド -機動救急警察-,4
3,C16039,フューチャーカード バディファイト DDD,4
4,C16067,BORUTO -ボルト- NARUTO NEXT GENERATIONS,3


最も多くの原作者と紐付けられているのは、`ぼのぼの[新]`、`てさぐれ! 部活もの すぴんおふ プルプルんシャルム と遊ぼう`、`トミカハイパーレスキュー ドライブヘッド -機動救急警察-`、`フューチャーカード バディファイト DDD`でした。

`ぼのぼの[新]`について見てみましょう。

In [46]:
# `acid`が`C16041`であるレコードを`df_ac_crt`から抽出
df_ac_crt[df_ac_crt["acid"] == "C16041"]

,acid,acname,asid,n_ae,first_date,last_date,crtid,crtname
881,C16041,ぼのぼの[新],C1491,38,2016-04-02,2016-12-24,ACRT00153,いがらしみきお
882,C16041,ぼのぼの[新],C1491,38,2016-04-02,2016-12-24,ACRT00270,アイ・エム・オー
883,C16041,ぼのぼの[新],C1491,38,2016-04-02,2016-12-24,ACRT00321,オフィス・コウキ
884,C16041,ぼのぼの[新],C1491,38,2016-04-02,2016-12-24,ACRT01219,竹書房


[Wikipedia](https://ja.wikipedia.org/wiki/%E3%81%BC%E3%81%AE%E3%81%BC%E3%81%AE_(2016%E5%B9%B4%E3%81%AE%E3%83%86%E3%83%AC%E3%83%93%E3%82%A2%E3%83%8B%E3%83%A1) によると、原作協力として、アイ・エム・オー、オフィス・コウキ、竹書房が携わっていたようです。

### `asid`列の深堀り

`asid`（アニメシリーズID）について分析してみましょう。

In [47]:
# `asid`ごとにユニークな`crtid`の数を集計し、結果の統計情報を取得
df_ac_crt.groupby("asid")["crtid"].nunique().describe().reset_index()

,index,crtid
0,count,896.000000
1,mean,1.380580
2,std,0.638803
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,2.000000
7,max,5.000000


一つのアニメシリーズに対して、平均で1.4人、最大で5人の原作者が紐付けられていることがわかります。
では、多くの原作者と紐づけられているアニメシリーズを確認してみましょう。

In [48]:
# df_ac_crtデータフレームを'first_date'列で昇順に並び替えて、'asid'でグループ化
# 次に、各グループにおける'crtid'（原作者ID）と'acid'（アニメ作品ID）のユニークな値の数を計算し、
# さらにそのグループの最初の'acname'（アニメ作品名）を取得
df_tmp = (
    df_ac_crt.sort_values("first_date")
    .groupby("asid")
    .agg({"crtid": "nunique", "acid": "nunique", "acname": "first"})
)

# 集計したデータフレームdf_tmpを'crtid'（原作者IDのユニークな数）で降順に並び替え
# そして、その結果の上位5行を表示して、最も多様な原作者と紐づいているasidを確認
df_tmp.sort_values("crtid", ascending=False).head()

,crtid,acid,acname
asid,,,
C2119,5,2,南海奇皇[第1期]
C3797,5,5,Fate/kaleid liner プリズマ ★イリヤ 2ｗｅｉ!
C3174,5,3,THE iDOLM＠STER シンデレラガールズ［第１期］
C5766,5,4,フューチャーカード バディファイト
C1240,4,1,トミカハイパーレスキュー ドライブヘッド -機動救急警察-


`acid`で集計した場合と結果が異なり、`南海奇皇`、`Fate/kaleid liner プリズマ ★イリヤ`、`THE iDOLM＠STER シンデレラガールズ`、そして`フューチャーカード`シリーズが最も多くの原作者と紐づいていることがわかりました。

### `crtid`、`crtname`列の深掘り

`crtid`（原作者ID）と`crtname`（原作者名）について分析してみましょう。

まず、`crtid`と`crtname`の対応関係を確認します。

In [49]:
# `crtid`単位で`crtname`のユニーク数を集計し、その統計情報を取得
df_ac_crt.groupby("crtid")["crtname"].nunique().describe().reset_index()

,index,crtname
0,count,1056.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`crtid`に対して、必ず一つの`crtname`が紐付いていることが確認できました。 では、その逆はどうでしょうか。

In [50]:
# `crtname`単位で`crtid`のユニーク数を集計し、その統計情報を取得
df_ac_crt.groupby("crtname")["crtid"].nunique().describe().reset_index()

,index,crtid
0,count,1056.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


逆についても同様でした。つまり、`crtid`と`crtname`が一対一で対応付けられていると言えます。

次に、一人の原作者あたり、どの程度のアニメ作品と対応づけられているか分析します。

In [51]:
# `crtid`および`crtname`単位で、紐付けられている`acid`のユニーク数を集計
# その結果の基礎統計情報を取得
df_ac_crt.groupby(["crtid", "crtname"])["acid"].nunique().describe().reset_index()

,index,acid
0,count,1056.000000
1,mean,1.414773
2,std,1.280673
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,29.000000


原作者一人に対し、平均で約1.4、最大で約29のアニメ作品が紐付けられていることがわかります。`acid`数が多い順に`crtname`を集計してみます。

In [52]:
# `crtname`単位で、紐付けられている`acid`のユニーク数を集計
# 結果を降順にソートし、上位10名の原作者とその紐づくアニメ作品数を抽出
df_ac_crt.groupby("crtname")["acid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,crtname,acid
0,矢立肇,29
1,富野由悠季,10
2,尼子騒兵衛,10
3,ブシロード,9
4,ルーツ,9
5,Piyo,8
6,タツノコプロ,7
7,武内直子,6
8,サンリオ,6
9,あかほりさとる,6


`矢立肇`さんが最も多く、29の`acid`と紐付けられていることがわかります。
ただし、`矢立肇`は特定の個人を指さず、サンライズの企画スタッフの共用ペンネームであることに注意が必要です。

また、このデータにおいては、原作者名（`crtname`）には社名も入りうることも頭の片隅においておくべきでしょう。

### `n_ae`列の深掘り

`n_ae`（アニメ各話数）について分析してみましょう。

まずは、原作者あたりの`n_ae`について記述統計情報を確認します。

In [53]:
# `crtname`単位での`n_ae`の合計を集計
# その集計結果の記述統計量を算出
df_ac_crt.groupby("crtname")["n_ae"].sum().describe().reset_index()

,index,n_ae
0,count,1056.000000
1,mean,39.488636
2,std,77.386859
3,min,1.000000
4,25%,12.000000
5,50%,24.000000
6,75%,38.000000
7,max,1267.000000


一人の原作者あたり、平均約39話のアニメ各話と紐付けられていることがわかります。

それでは、合計`n_ae`数が多い原作者を抽出してみましょう。

In [54]:
# `crtname`単位での`n_ae`の合計を集計し、降順にソート
# 上位10人の原作者を抽出
df_ac_crt.groupby("crtname")["n_ae"].sum().sort_values(
    ascending=False
).reset_index().head(10)

,crtname,n_ae
0,長谷川町子,1267
1,矢立肇,997
2,やなせたかし,958
3,尼子騒兵衛,817
4,サンリオ,644
5,ブシロード,363
6,犬丸りん,343
7,手塚治虫,328
8,富野由悠季,320
9,タツノコプロ,316


`サザエさん`の`長谷川町子`さん、サンライズの企画スタッフの共用ペンネームである`矢立肇`さん、`アンパンマン`の`やなせたかし`さん、`忍たま乱太郎`の`尼子騒兵衛`さん等が並びます。

しかし、`an_ae.csv`で最も多くの各話数を記録されていた`クレヨンしんちゃん`の原作者である`臼井儀人`さんの名前がないことが気になります。

In [55]:
# `df_ac_crt`から`クレヨンしんちゃん`のレコード数をカウント
(df_ac_crt["acname"] == "クレヨンしんちゃん").sum()

0

前述した通り、`an_ac_crt.csv`の`acid`のカバー率は約30%にとどまることを再認識しておきましょう。

### `first_date`、`last_date`列の深掘り

`first_date`（最初の放送日）と`last_date`（最後の放送日）について分析してみましょう。

まず、分析用に`first_date`と`last_date`の型を変更します。

In [56]:
# `df_ac_crt`の`first_date`列をdatetime型に変換
df_ac_crt["first_date"] = pd.to_datetime(df_ac_crt["first_date"])
# `df_ac_crt`の`last_date`列をdatetime型に変換
df_ac_crt["last_date"] = pd.to_datetime(df_ac_crt["last_date"])

原作者あたりの活動期間の長さ（最も古い`first_date`から最も新しい`last_date`までの日数）を集計し、特に活動期間が長い原作者を列挙します。

In [57]:
# `df_ac_crt`から、原作者(`crtname`)ごとの最初の放送日(`first_date`)
# と最後の放送日(`last_date`)を集計
df_tmp = (
    df_ac_crt.groupby("crtname")[["first_date", "last_date"]]
    .agg({"first_date": "min", "last_date": "max"})
    .reset_index()
)

# 各原作者の活動期間を計算
df_tmp["duration"] = df_tmp["last_date"] - df_tmp["first_date"]

# 活動期間が長いトップ10の原作者を取得
df_tmp.sort_values(by="duration", ascending=False).head(10)

,crtname,first_date,last_date,duration
645,手塚治虫,1963-01-01,2017-07-08,19912 days
384,モンキー・パンチ,1971-10-24,2016-03-18,16217 days
864,矢立肇,1979-04-07,2016-12-25,13777 days
978,赤塚不二夫,1990-04-21,2016-12-13,9733 days
725,森下裕美,1991-04-11,2017-09-19,9658 days
947,藤子不二雄Ⓐ,1991-03-12,2017-06-19,9596 days
785,池田あきこ,1992-03-31,2016-12-31,9041 days
508,吉岡平,1993-01-25,2017-09-26,9010 days
751,横山光輝,1991-10-18,2016-03-26,8926 days
761,武内直子,1992-03-07,2016-06-27,8878 days


## `an_ac_act.csv`の基礎分析

`an_ac_act.csv`は、アニメ作品と声優に関する情報を一つに集約したファイルです。
次の列を持ちます：

- `acid`：Anime Collection ID。アニメ作品ID
- `acname`：Anime Collection NAME。アニメ作品名
- `asid`：Anime Series ID。アニメシリーズID
- `n_ae`：アニメ各話の合計数
- `first_date`：最初の放送日
- `last_date`：最後の放送日
- `actid`：ACTor ID。声優ID
- `actname`：ACTor NAME。声優名
- `wiki_size`：当該声優のウィキペディアのページサイズ
- `gender`：当該声優の性別

In [58]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ac_act = pd.read_csv(DIR_IN / FN_AC_ACT)

### 全体像の把握

`df_ac_act`を概観しましょう。

In [59]:
# df_ac_actデータフレームの先頭5行を転置して表示
df_ac_act.head().T

,0,1,2,3,4
acid,C10001,C10001,C10001,C10001,C10001
acname,ギャラクシー エンジェル,ギャラクシー エンジェル,ギャラクシー エンジェル,ギャラクシー エンジェル,ギャラクシー エンジェル
asid,C2483,C2483,C2483,C2483,C2483
n_ae,24,24,24,24,24
first_date,2001-04-08,2001-04-08,2001-04-08,2001-04-08,2001-04-08
last_date,2001-09-30,2001-09-30,2001-09-30,2001-09-30,2001-09-30
actid,ACT00102,ACT05700,ACT06001,ACT01887,ACT02359
actname,かないみか,保村真,吉野裕行,山口眞弓,新谷良子
wiki_size,116003.0,45464.0,149454.0,19635.0,73259.0
gender,female,male,male,female,female


形状を確認してみましょう。

In [60]:
# df_ac_actデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ac_act.shape

(30492, 10)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [61]:
# df_ac_actデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# その後、agg()メソッドを使用して、各列における欠損値の合計と平均を計算
df_ac_act.isna().agg([sum, "mean"]).T

,sum,mean
acid,0.0,0.000000
acname,0.0,0.000000
asid,67.0,0.002197
n_ae,0.0,0.000000
first_date,0.0,0.000000
last_date,0.0,0.000000
actid,0.0,0.000000
actname,0.0,0.000000
wiki_size,0.0,0.000000
gender,0.0,0.000000


`asid`にわずかな欠損が見られます。

記述統計情報を確認してみましょう。

In [62]:
# df_ac_actデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の中央値、平均、標準偏差などの基本的な統計量を表示
df_ac_act.describe()

,n_ae,wiki_size
count,30492.000000,30492.000000
mean,29.810245,120299.661124
std,71.504224,95596.818847
min,1.000000,84.000000
25%,12.000000,40113.000000
50%,13.000000,93599.000000
75%,26.000000,183982.000000
max,1926.000000,393910.000000


ユニークな値の数を確認してみましょう。

In [63]:
# df_ac_actデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を"nunique"とする
df_ac_act.nunique().reset_index(name="nunique")

,index,nunique
0,acid,2845
1,acname,2842
2,asid,1977
3,n_ae,150
4,first_date,1331
5,last_date,1478
6,actid,2998
7,actname,2998
8,wiki_size,2932
9,gender,2


行数より`acid`および`actid`のユニーク数が少ないので、それぞれ単独では重複が存在することがわかります。

In [64]:
# `acid`と`crtid`の組み合わせで重複する行の数をカウント
df_ac_act.duplicated(subset=["acid", "actid"]).sum()

0

`acid`と`actid`の組合せに関しては重複がないことがわかります。

`acid`のカバー率（`an_ae.csv`の`acid`のうち、`an_ac_act.csv`に存在するものの割合）を確認します。

(an-eda-act-coverage)=

In [65]:
# `df_ac_act`と`df_ae`で共通の`ccid`の数を取得
n_acid_w_act = len(set(df_ac_act["acid"].unique()) & set(df_ae["acid"].unique()))

# `df_ae`内のユニークな`acid`の数を取得
n_acid = df_ae["acid"].nunique()

# カバー率を計算して表示
print(f"{n_acid_w_act} / {n_acid} = {n_acid_w_act / n_acid}")

2845 / 3637 = 0.782238108331042


全体の約78%、2845のアニメ作品に対して声優が対応付けられていることがわかります。

以降、カラムごとに少し踏み込んだ分析を行います。

### `acid`、`acname`列の分析

まず、`acid`（アニメ作品ID）と`acname`（アニメ作品名）について分析してみましょう。

In [66]:
# acid、acnameに紐づいている声優数を集計
df_ac_act.groupby(["acid", "acname"])["actid"].nunique().describe().reset_index()

,index,actid
0,count,2845.000000
1,mean,10.717750
2,std,5.068738
3,min,1.000000
4,25%,7.000000
5,50%,11.000000
6,75%,15.000000
7,max,22.000000


一つのアニメ作品に対して、平均で約11人、最大で22人の声優が紐付けられていることがわかります。

In [67]:
# 紐づいている声優数が多いアニメ作品を抽出
df_ac_act.groupby(["acid", "acname"])["actid"].nunique().sort_values(
    ascending=False
).reset_index().head()

,acid,acname,actid
0,C14818,カーニヴァル,22
1,C15870,甲鉄城のカバネリ,22
2,C14716,キングダム,21
3,C13851,神様のメモ帳 It's the only NEET thing to do.,21
4,C14833,キングダム[第2期],21


最も多くの声優と紐付けられているのは、`カーニヴァル`と`甲鉄城のカバネリ`でした。

`カーニヴァル`について見てみましょう。

In [68]:
# acidがC14818のデータを抽出し、見やすさのため特定の列のみ表示
df_ac_act[df_ac_act["acid"] == "C14818"][["acname", "actname", "gender"]]

,acname,actname,gender
19711,カーニヴァル,下野紘,male
19712,カーニヴァル,中村悠一,male
19713,カーニヴァル,五十嵐裕美,female
19714,カーニヴァル,佐藤聡美,female
19715,カーニヴァル,保志総一朗,male
19716,カーニヴァル,入野自由,male
19717,カーニヴァル,前野智昭,male
19718,カーニヴァル,喜多村英梨,female
19719,カーニヴァル,宮野真守,male
19720,カーニヴァル,小野大輔,male


様々な声優が紐付けられていることがわかります。

### `asid`列の深堀り

`asid`（アニメシリーズID）について分析してみましょう。

In [69]:
# `asid`ごとにユニークな`actid`の数を集計し、結果の統計情報を取得
df_ac_act.groupby("asid")["actid"].nunique().describe().reset_index()

,index,actid
0,count,1977.000000
1,mean,12.465352
2,std,8.874857
3,min,1.000000
4,25%,7.000000
5,50%,12.000000
6,75%,16.000000
7,max,165.000000


アニメシリーズ一つあたり、平均で約12人、最大で165人の声優が紐づけられていることがわかります。

In [70]:
# まず、df_ac_actデータフレームを'first_date'列で昇順に並び替えて、'asid'でグループ化
# 次に、各グループにおける'actid'のユニークな値の数（活動の種類の数）と、
# 'acname'の最初の値（最初の活動名）を集計し、グループ化を解除して新しいデータフレームdf_tmpを作成
df_tmp = (
    df_ac_act.sort_values("first_date")
    .groupby("asid")
    .agg({"actid": "nunique", "acname": "first"})
    .reset_index()
)

# 次に、集計したデータフレームdf_tmpを'actid'（活動のユニークな数）で降順に並び替え
# そして、その結果の上位5行を表示して、最も活動の種類が多い上位のasidを確認
df_tmp.sort_values("actid", ascending=False).head()

,asid,actid,acname
391,C2833,165,ふたりはプリキュア
1959,C6820,91,超生命体 トランスフォーマー ビーストウォーズ メタルス
169,C2454,89,爆転シュート ベイブレード
301,C2650,84,デュエル・マスターズ
728,C3397,79,バトルスピリッツ少年突破バシン


最も多くの声優と紐づいているのは、`プリキュア`シリーズであることがわかりました。

### `actid`、`actname`列の深掘り

`actid`（声優ID）と`actname`（声優名）について分析してみましょう。

まずは、`actid`と`actname`の対応関係を確認します。

In [71]:
# `actid`ごとにユニークな`actname`の数を集計し、結果の統計情報を取得
df_ac_act.groupby("actid")["actname"].nunique().describe().reset_index()

,index,actname
0,count,2998.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`actid`に対して、必ず一つの`actname`が対応していることがわかります。

In [72]:
# `actname`ごとにユニークな`actid`の数を集計
# その後、結果の統計情報を取得
df_ac_act.groupby("actname")["actid"].nunique().describe().reset_index()

,index,actid
0,count,2998.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


逆についても同様です。つまり、`actid`と`actname`が一対一で対応付けられていることが確認できました。

次は、声優一人あたりに対応づけられているアニメ作品数を調べてみましょう。

In [73]:
# `actid`と`actname`の組合せごとにユニークな`acid`の数を集計
# その後、結果の統計情報を取得
df_ac_act.groupby(["actid", "actname"])["acid"].nunique().describe().reset_index()

,index,acid
0,count,2998.000000
1,mean,10.170781
2,std,19.762846
3,min,1.000000
4,25%,1.000000
5,50%,3.000000
6,75%,9.000000
7,max,181.000000


声優一人に対し、平均で約10、最大で181のアニメ作品が紐付けられていることがわかります。`acid`数が多い順に`actname`を集計してみます。

In [74]:
# `df_ac_act`で各声優名(`actname`)ごとにユニークなアニメ作品(`acid`)の数を集計
# その後、降順にソートして上位10件を表示
df_ac_act.groupby("actname")["acid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,actname,acid
0,沢城みゆき,181
1,櫻井孝宏,180
2,子安武人,175
3,能登麻美子,158
4,釘宮理恵,156
5,福山潤,154
6,堀江由衣,149
7,花澤香菜,142
8,浪川大輔,139
9,川澄綾子,138


このデータにおいては、`沢城みゆき`さんが最も多く、181のアニメ作品と紐付けられていることがわかります。

### `n_ae`列の深掘り

`n_ae`（アニメ作品の合計各話数）について分析してみましょう。
まず、声優一人に対して対応づけられている各話数を集計してみます。

In [75]:
# `actid`と`actname`の組合せごとに`n_ae`の合計数を集計
# その後、結果の統計情報を取得
df_ac_act.groupby(["actid", "actname"])["n_ae"].sum().describe().reset_index()

,index,n_ae
0,count,2998.000000
1,mean,303.193462
2,std,584.690032
3,min,1.000000
4,25%,26.000000
5,50%,70.000000
6,75%,262.500000
7,max,5468.000000


声優一人あたり、平均約303話、最大5468話のアニメ各話と紐付けられていることがわかります。

それでは、合計`n_ae`数が多い声優を抽出してみましょう。

In [76]:
# `df_ac_act`で声優名(`actname`)ごとにアニメ各話数(`n_ae`)の合計値を集計
# その後、降順にソートして上位10件を表示
df_ac_act.groupby("actname")["n_ae"].sum().sort_values(
    ascending=False
).reset_index().head(10)

,actname,n_ae
0,藤原啓治,5468
1,こおろぎさとみ,4546
2,川澄綾子,4429
3,石田彰,4389
4,山口勝平,4328
5,子安武人,4255
6,一龍斎貞友,4247
7,釘宮理恵,4107
8,櫻井孝宏,3908
9,三石琴乃,3739


最も多くのアニメ各話と紐付けられている声優は、`藤原啓治`さんでした。
ここでふと気になったので、最も多くのアニメ各話が存在する`クレヨンしんちゃん`と紐付けられている声優を調べてみます。

In [77]:
# `df_ac_crt`から`acname`が`クレヨンしんちゃん`である行を抽出
# 見やすさのため、特定の列のみ表示
df_ac_act[df_ac_act["acname"] == "クレヨンしんちゃん"][["acname", "actname", "gender"]]

,acname,actname,gender
29115,クレヨンしんちゃん,こおろぎさとみ,female
29116,クレヨンしんちゃん,ならはしみき,female
29117,クレヨンしんちゃん,一龍斎貞友,female
29118,クレヨンしんちゃん,三石琴乃,female
29119,クレヨンしんちゃん,佐藤智恵,female
29120,クレヨンしんちゃん,佳川紘子,female
29121,クレヨンしんちゃん,富沢美智恵,female
29122,クレヨンしんちゃん,川澄綾子,female
29123,クレヨンしんちゃん,林玉緒,female
29124,クレヨンしんちゃん,桜井敏治,male


上位3名まで`クレヨンしんちゃん`に対応づけられている声優であることがわかりました。

### `first_date`、`last_date`列の深掘り

`first_date`（最初の放送日）と`last_date`（最後の放送日）について分析してみましょう。

まず、分析用に`first_date`と`last_date`の型を変更します。

In [78]:
# `df_ac_act`の`first_date`列をdatetime型に変換
df_ac_act["first_date"] = pd.to_datetime(df_ac_act["first_date"])
# `df_ac_act`の`last_date`列をdatetime型に変換
df_ac_act["last_date"] = pd.to_datetime(df_ac_act["last_date"])

声優あたりの活動期間の長さ（最も古い`first_date`から最も新しい`last_date`までの日数）を集計します。

In [79]:
# `df_ac_crt`から、声優(`actname`)ごとの最初の放送日(`first_date`)
# と最後の放送日(`last_date`)を集計
df_tmp = (
    df_ac_act.groupby("actname")[["first_date", "last_date"]]
    .agg({"first_date": "min", "last_date": "max"})
    .reset_index()
)

# 各声優の活動期間を計算
df_tmp["duration"] = df_tmp["last_date"] - df_tmp["first_date"]

# 活動期間が長い10名の声優を表示
df_tmp.sort_values(by="duration", ascending=False).head(10)

,actname,first_date,last_date,duration
798,増岡弘,1963-11-25,2016-12-25,19389 days
870,大竹宏,1963-11-25,2015-10-24,18961 days
2208,矢島正明,1963-01-01,2014-04-07,18724 days
501,内海賢二,1963-11-25,2011-04-08,17301 days
485,八奈見乗児,1963-11-25,2010-11-21,17163 days
2390,納谷悟朗,1971-10-24,2016-12-25,16499 days
837,大塚周夫,1971-10-24,2016-01-03,16142 days
2000,清水マリ,1963-01-01,2006-01-10,15715 days
2410,緒方賢一,1974-10-06,2017-09-30,15700 days
2984,麻生美代子,1974-01-06,2016-12-25,15694 days


このデータにおいて最も活動期間が長いのは、`増岡弘`さん（`19389`日は約53年に相当します！）であることがわかりました。

### `wiki_size`列の深掘り

`wiki_size`（声優のWikipediaページのサイズ）について分析してましょう。

In [80]:
# 声優ごとに、それぞれのWikipediaページのサイズを集計
df_tmp = df_ac_act.groupby("actname")["wiki_size"].first().reset_index()

# Wikipediaページサイズの基本統計量を表示
df_tmp["wiki_size"].describe().reset_index()

,index,wiki_size
0,count,2998.000000
1,mean,41377.184456
2,std,53608.653558
3,min,84.000000
4,25%,8906.250000
5,50%,21203.000000
6,75%,51036.500000
7,max,393910.000000


声優一人あたり、平均40967バイト、最大393910バイトの情報がWikipediaに書き込まれていたことがわかります。

次に、Wikipediaの情報量が大きい声優を抽出してみましょう。

In [81]:
# Wikipediaページサイズが大きい上位10人の声優を表示
df_tmp.sort_values("wiki_size", ascending=False).head(10)

,actname,wiki_size
2446,花澤香菜,393910.0
507,内田真礼,388653.0
1502,早見沙織,364131.0
1769,森川智之,360890.0
1647,松岡禎丞,356962.0
1924,沢城みゆき,354094.0
1863,水樹奈々,336020.0
1594,杉田智和,330195.0
838,大塚明夫,328094.0
1839,櫻井孝宏,325313.0


このデータにおいては、最もWikipediaの情報量が多いのは`花澤香菜`さんであることがわかりました。

### `gender`列の深掘り

`gender`（声優の性別）[^gender]について分析してみましょう。

[^gender]: この分析では、性別を男性と女性という形で区分けしていますが、これは単に分析の手法上の選択です。筆者は、性別の多様性を深く尊重しており、多様な性自認や表現を含めた性のスペクトラム全体を認識しています。この文脈での性別の使用は、あくまで分析の枠組みを単純化するためのものであり、特定の性別認識の排除や無視を意図するものではありません。

まず、各性別の声優数を集計します。

In [82]:
# 各性別（`gender`）ごとに、ユニークな声優（`actid`）の数を集計
df_ac_act.groupby("gender")["actid"].nunique().reset_index()

,gender,actid
0,female,1664
1,male,1334


このデータにおいては、女性声優の方が男性声優より多く格納されていることがわかりました。

次に、紐付けられている作品数を`gender`別に集計します。

In [83]:
# 各性別（`gender`）ごとに、合計行数をカウント
df_ac_act.value_counts("gender").reset_index()

,gender,count
0,female,16486
1,male,14006


このデータにおいては、女性声優が合計16486作品と、男性声優が合計14006作品と紐づけられています。

念のため、同一`actid`に対して複数の`gender`が紐付けられていないか確認します。

In [84]:
# 各声優（actid）に対して、性別（gender）が一意であることを確認
assert df_ac_act.groupby("actid")["gender"].nunique().max() == 1

では、`gender`ごとに、紐付けられているアニメ作品数の分布に違いがあるか確認してみましょう。

In [86]:
# 各声優（actid）と性別（gender）ごとに、関与したユニークなアニメ作品（acid）の数を集計
df_tmp = (
    df_ac_act.groupby(["actid", "gender"])["acid"].nunique().reset_index(name="n_ac")
)

# 上で集計したアニメ作品数を、性別ごとに基本統計量を計算する
df_tmp.groupby("gender")["n_ac"].describe().reset_index()

,gender,count,mean,std,min,25%,50%,75%,max
0,female,1664.0,9.907452,18.927906,1.0,1.0,3.0,9.0,181.0
1,male,1334.0,10.499250,20.759826,1.0,1.0,3.0,9.0,180.0


レコード数（`count`）こそ違いがありますが、紐付けられているアニメ作品数（`n_ac`）の分布に関しては、`gender`間に違いがないように見えます[^xi]。

[^xi]: 性別の違いが出演作品数の違いに与える影響がないことを確かめるためには、統計的検定（例えば、カイ二乗検定等）を実施することが考えられます。本書のスコープから外れるため割愛しましたが、興味のある方は調べてみると良いかもしれません。

## まとめ

ここでは、`an_ae.csv`、`an_ac_crt.csv`、そして`an_ac_act.csv`について基礎分析を行いました。 重要な発見や示唆がたくさんありますので、もう一度復習しておきましょう。

まず大前提として、本書で扱うアニメデータは、 [MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)として公開されているものをGitHub経由で取得し、[所定の前処理](../appendix/an_preprocess.ipynb)を行ったものです。
放送日が **1963年1月1日から2017年10月15日** までの **テレビレギュラー放送** を対象としています。
特に、 **上記の対象外で活躍した原作者や声優が過小に評価され得る** という点は、常に意識しておくべきです。

`an_ae.csv`は、アニメ各話についてまとめたファイルです。今後分析で用いる際は、以下に特に注意するべきでしょう：

- `aename`に一定程度の欠損がある
- `asid`と`aeno`に僅かな欠損がある
- `date`は大規模な欠損があることが想定される
    - 例：1967〜1970年、1973年、1976〜1978年、1981〜1989年は一切アニメ各話データが登録されていない。原因の一つとして、元データにおける放送日情報の欠損が考えられる
- `acid`と`acname`が一対一で対応していないケースが少数存在する
    - 例：`100％ パスカル先生`。同一`acname`に対して複数の`acid`が紐づいている。同名作品でありながら放送局や形態が異なるシーズンが存在することが原因と考えられる
- アニメシリーズによって、期やクールごとに`acid`を分ける作品とそうでない作品が存在する
    - 例：`忍たま乱太郎`シリーズは期ごとに`acid`を分けているが、`クレヨンしんちゃん`は一つの`acid`を用いている

`an_ac_crt.csv`は、アニメ作品と原作者についてまとめたファイルです。今後分析で用いる際は、以下に特に注意するべきでしょう：

- `acid`と`crtid`の組合せが主キーとなっているため、それぞれ単独では重複する行が存在する
- `an_ae.csv`中の`acid`のうち、`an_ac_crt.csv`に含まれるものは約30%に過ぎない
    - `an_ae.csv`における`acid`数：3637
    - `an_ac_crt.csv`における`acid`数：1107
- `crtname`には、個人名だけでなく、社名が入ることがある
    - 例：`ブシロード`、`タツノコプロ`
 - `first_date`および`last_date`は、あくまでもこのデータの対象期間中の最初と最後の放送日

`an_ac_act.csv`は、アニメ作品と声優の関係をまとめたファイルです。今後分析で用いる際は、以下に特に注意するべきでしょう：

- [2023年2月24日にPetScanから以下の条件で取得した声優データ](an-preprocess-wikipedia)を元にしている
    - 言語：ja
    - プロジェクト：wikipedia
    - カテゴリ深度：1
    - カテゴリ：
        - 日本の男性声優
        - 日本の女性声優
- `acid`と`actid`が主キーとなっているため、それぞれ単独では重複する行が存在する
- `an_ae.csv`中の`acid`のうち、`an_ac_act.csv`に含まれるものは約78%
- `first_date`および`last_date`は、あくまでもこのデータの対象期間中の最初と最後の放送日